In [14]:
import backtrader as bt
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
import backtrader.feeds as btfeeds


In [16]:
#Defining the moving average strategy
class TrimaStrategy(bt.Strategy):
    params = dict(tfast=5, tslow=35)
    
    def log(self, message, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print("%s %s" %(dt.isoformat(),message))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.price = None
        self.commission = None
        trima_fast = bt.talib.TRIMA(close=self.dataclose, timeperiod=self.params.tfast)
        trima_slow = bt.talib.TRIMA(close=self.dataclose, timeperiod=self.params.tslow)
        volume_indicator = bt.talib.AD(high=self.datas[0].high, low=self.datas[0].low,
                                      close=self.datas[0].close, volume=self.datas[0].volume)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            #Updating and printing the values if the order has gone through
            if order.isbuy():
                self.log('Stock Purchased, Price: %.2f,\tCost: %.2f,\tCommission: %.2f' 
                         %(order.executed.price, order.executed.value, order.executed.comm))
                self.price = order.executed.price
                self.commision = order.executed.comm
                
            else:
                self.log('Stock Sold, Price: %.2f,\tCost: %.2f,\tCommission: %.2f'
                         %(order.executed.price, order.executed.value, order.executed.comm))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        #Any of the above two conditions, there is no pending order and an order can be placed
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %(trade.pnl, trade.pnlcomm))
        
    def next(self):
        #Uncomment the next statement to log all the closing prices  of the series
        #self.log('Close, %.2f' % self.dataclose[0])
        
        if self.order:
            return
        
        if not self.position:  #if not in the market
            if (trima_fast-trima_slow) > 0.7:       #checking for the signal using the values given
                if volume_indicator>0.0:            #checking the volume crossover condition
                    self.log('Buy Order Placed: %.2f' %self.dataclose[0])
                    #placing a stop loss order
                    self.order = self.buy(data=self.datas[0], price=self.dataclose*0.8, exectype=Order.Stop)
                    self.price = self.order.price
                    self.comm = self.order.comm              
            
            
            if self.order != None:
                if self.order.value > 1.3*self.order.price:
                    self.log('Price target breached, Sell order placed: %.2f' %self.dataclose[0])
                    self.order = self.close()                
                
                elif (trima_fast-trima_slow < 0.4) :
                    self.log('Exit Signal Generated, Sell Order Placed: %.2f' %self.dataclose[0])
                    self.order = self.close()        
                    #DO NOT use sell here, it will sell the stock even if it is not bought

                #Selling an existing stock, just before the market closes
                if (self.data.datetime()>=datetime.time(15,15)):
                    self.log('Near EoD, Sell Order Placed: %.2f' %self.dataclose[0])
                    self.order = self.close()

In [18]:
if __name__ == '__main__':
    
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TrimaStrategy)
    
    #Data path specific to my computer
    datapath = '/home/aviral/Downloads/NIFTY_F1.xlsb'
    data = btfeeds.GenericCSVData(
        dataname = datapath,
        fromdate = datetime.datetime(2013, 1, 1),
        todate = datetime.datetime(2013, 12, 31),

        nullvalue = 0.0,

        dtformat = ('%d/%m/%Y'),
        tmformat = ('%H:%M:%S'),

        datetime=0,
        time=1,
        open=2,
        high=3,
        low=4,
        close=5,
        volume=6,
        openinterest=-1
    )
    
    cerebro.broker.setcash(100000.0)
    cerebro.adddata(data)
    cerebro.broker.setcommission(commission=0.001)
    
    initial = cerebro.broker.getvalue()
    print('Initial Portfolio Value: %.2f' %initial)

    cerebro.run()

    #While calculating the final portfolio value, the current closing price of stock is taken, if not sold
    final =  cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' %final)
    print('Net profit/loss on the trade: %.2f' %(final-initial))



Initial Portfolio Value: 100000.00


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xeb in position 17: invalid continuation byte